<a href="https://colab.research.google.com/github/nomi181472/MS-DS-Practice/blob/main/ann/gan/Simple_Gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image
import numpy as np

import matplotlib.pyplot as plt
from datetime import datetime
import os

In [2]:
transform=transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5,),std=(0.5,),),

    ]
)

In [3]:
train_dataset=torchvision.datasets.MNIST(
    root=",",
    train=True,
    transform=transform,
    download=True
)

100%|██████████| 9912422/9912422 [00:00<00:00, 99181776.26it/s]


Extracting ,/MNIST/raw/train-images-idx3-ubyte.gz to ,/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 89267276.22it/s]


Extracting ,/MNIST/raw/train-labels-idx1-ubyte.gz to ,/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 27649077.47it/s]


Extracting ,/MNIST/raw/t10k-images-idx3-ubyte.gz to ,/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 5249525.70it/s]

Extracting ,/MNIST/raw/t10k-labels-idx1-ubyte.gz to ,/MNIST/raw



In [4]:
test_dataset=torchvision.datasets.MNIST(
    root=",",
    train=False,
    transform=transform,
    download=True
)

In [5]:
len(train_dataset)

60000

In [6]:
batch_size=128
data_loader=torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)

In [7]:
D=nn.Sequential(
    nn.Linear(784,512),
    nn.LeakyReLU(0.2),
    nn.Linear(512,256),
    nn.LeakyReLU(0.2),
    nn.Linear(256,1)
)
D

Sequential(
  (0): Linear(in_features=784, out_features=512, bias=True)
  (1): LeakyReLU(negative_slope=0.2)
  (2): Linear(in_features=512, out_features=256, bias=True)
  (3): LeakyReLU(negative_slope=0.2)
  (4): Linear(in_features=256, out_features=1, bias=True)
)

In [8]:
latin_dim=10
G=nn.Sequential(
    nn.Linear(latin_dim,256),
    nn.LeakyReLU(-.2),
    nn.BatchNorm1d(256,momentum=0.7),

    nn.Linear(256,512),
    nn.LeakyReLU(0.2),
    nn.BatchNorm1d(512,momentum=0.7),


    nn.Linear(512,1024),
    nn.LeakyReLU(),
    nn.BatchNorm1d(1024,momentum=0.7),

    nn.Linear(1024,784),
    nn.Tanh()


)
G

Sequential(
  (0): Linear(in_features=10, out_features=256, bias=True)
  (1): LeakyReLU(negative_slope=-0.2)
  (2): BatchNorm1d(256, eps=1e-05, momentum=0.7, affine=True, track_running_stats=True)
  (3): Linear(in_features=256, out_features=512, bias=True)
  (4): LeakyReLU(negative_slope=0.2)
  (5): BatchNorm1d(512, eps=1e-05, momentum=0.7, affine=True, track_running_stats=True)
  (6): Linear(in_features=512, out_features=1024, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): BatchNorm1d(1024, eps=1e-05, momentum=0.7, affine=True, track_running_stats=True)
  (9): Linear(in_features=1024, out_features=784, bias=True)
  (10): Tanh()
)

In [9]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [10]:
D=D.to(device)
G=G.to(device)

In [11]:
criterion=nn.BCEWithLogitsLoss()
d_optim=torch.optim.Adam(D.parameters(),lr=0.0002,betas=(0.5,0.999))
g_optim=torch.optim.Adam(G.parameters(),lr=0.0002,betas=(0.5,0.999))

In [12]:
def scale_image(img):
  out= (img+1)/2
  return out

In [13]:
if not os.path.exists("gan_images"):
  os.makedirs("gan_images")

In [ ]:
ones_=torch.ones(batch_size,1).to(device)
zeros_=torch.zeros(batch_size,1).to(device)


d_losses=[]
g_losses=[]


for epoch in range(200):
  for inputs, _ in data_loader:

    n=inputs.size(0)
    inputs=inputs.reshape(n,784).to(device)
    ones=ones_[:n]
    zeros=zeros_[:n]

    #real
    real_outputs=D(inputs)
    d_loss_real=criterion(real_outputs,ones)

    #fake
    noise=torch.randn(n,latin_dim).to(device)
    fake_images=G(noise)
    fake_outputs=D(fake_images)
    d_loss_fake= criterion(fake_outputs,zeros)

    #gradient descent step

    d_loss=0.5 * (d_loss_real +d_loss_fake)
    d_optim.zero_grad()
    g_optim.zero_grad()
    d_loss.backward()
    d_optim.step()

    #train generator

    for _ in range(2):
      #fake images
      noise=torch.randn(n,latin_dim,).to(device)
      fake_images=G(noise)
      fake_outputs=D(fake_images)
      #reverse the labels
      g_loss=criterion(fake_outputs,ones)

      #gradient descent step
      d_optim.zero_grad()
      g_optim.zero_grad()
      g_loss.backward()
      g_optim.step()

    #save losses
    d_losses.append(d_loss.item())
    g_losses.append(g_loss.item())
  print(f"epoch:{epoch+1}/200 d_loss:{d_loss.item()}  g_loss: {g_loss.item()}")

  fake_images=fake_images.reshape(-1,1,28,28)
  save_image(scale_image(fake_images),f"gan_images/{epoch+1}.png")









epoch:1/200 d_loss:0.6876547336578369  g_loss: 0.6849592924118042
epoch:2/200 d_loss:0.6792889833450317  g_loss: 0.6885757446289062
epoch:3/200 d_loss:0.702298641204834  g_loss: 0.7209219932556152
epoch:4/200 d_loss:0.6913845539093018  g_loss: 0.706448495388031
epoch:5/200 d_loss:0.6924445033073425  g_loss: 0.6903718709945679
epoch:6/200 d_loss:0.6996534466743469  g_loss: 0.6983050107955933
epoch:7/200 d_loss:0.6976673603057861  g_loss: 0.7216713428497314
epoch:8/200 d_loss:0.6877574324607849  g_loss: 0.6999737620353699
epoch:9/200 d_loss:0.6972793936729431  g_loss: 0.6584462523460388
epoch:10/200 d_loss:0.6918821334838867  g_loss: 0.7415479421615601
epoch:11/200 d_loss:0.6989825367927551  g_loss: 0.6271547675132751
epoch:12/200 d_loss:0.6933650374412537  g_loss: 0.690377950668335
epoch:13/200 d_loss:0.6912143230438232  g_loss: 0.7164678573608398
epoch:14/200 d_loss:0.6956421136856079  g_loss: 0.7128937244415283
epoch:15/200 d_loss:0.6893370747566223  g_loss: 0.6889240741729736
epoch:1

In [ ]:
plt.plot(g_losses,label="g_losses")
plt.plot(d_losses,label="d_losses")
plt.legend()
plt.show()